In [ ]:
from google.colab import drive
drive.mount('/content/GoogleDrive', force_remount=True)
path = '/content/GoogleDrive/My Drive/ABSA'
%cd {path}
!ls

In [2]:
import pandas as pd
df_train = pd.read_csv('dataset/VLSP_Hotel/hotel_train.csv')
df_val = pd.read_csv('dataset/VLSP_Hotel/hotel_val.csv')
df_test = pd.read_csv('dataset/VLSP_Hotel/hotel_test.csv')
df_train.Review.head()

0    Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...
1    Địa điểm thuận tiện, trong vòng bán kính 1,5km...
2                            Phục vụ, view đẹp, vị trí
3               thuận tiện , sạch sẽ , vui vẻ hài lòng
4    Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...
Name: Review, dtype: object

# Preprocessing

In [3]:
!pip install -q emoji
!pip install -q vncorenlp
!git clone https://github.com/vncorenlp/VnCoreNLP.git

     |████████████████████████████████| 133kB 3.7MB/s 
     |████████████████████████████████| 2.7MB 4.0MB/s 
Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 212, done.
remote: Total 212 (delta 0), reused 0 (delta 0), pack-reused 212
Receiving objects: 100% (212/212), 214.21 MiB | 24.21 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Checking out files: 100% (33/33), done.


In [4]:
!rm -rf teencode.txt
!wget https://gist.githubusercontent.com/nguyenvanhieuvn/7d9441c10b3c2739499fc5a4d9ea06fb/raw/df939245b3e841b62af115be4dcb3516dadc9fc5/teencode.txt

--2021-07-03 17:16:15--  https://gist.githubusercontent.com/nguyenvanhieuvn/7d9441c10b3c2739499fc5a4d9ea06fb/raw/df939245b3e841b62af115be4dcb3516dadc9fc5/teencode.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5656 (5.5K) [text/plain]
Saving to: ‘teencode.txt’

teencode.txt        100%[===================>]   5.52K  --.-KB/s    in 0.001s  

2021-07-03 17:16:15 (4.25 MB/s) - ‘teencode.txt’ saved [5656/5656]



In [5]:
from preprocess import (
    remove_HTML, 
    convert_unicode, 
    # standardize_sentence_typing, 
    normalize_acronyms, 
    word_segmentation, # When use PhoBERT
    remove_unnecessary_characters
)

# Spell checking

In [6]:
from multiprocessing import dummy, cpu_count
from tqdm.notebook import tqdm
import requests

BASE_URL = 'https://viettelgroup.ai/nlp/api/v1/spell-checking'
TOKEN = '4m8bkBo0IO1HKC-QmF0Ygy01GqGdB-zBF4YwwBKDtbNnkfPmsr9jIURbnMI5SSI2'

In [7]:
def format_response(response, sentence): 
    length_diff = 0
    original_length = len(sentence.split())

    response = response.json()['result']['suggestions']
    if response == []: return sentence

    for word_suggestion in response:
        start, end, original, suggestion, _ = word_suggestion.values()
        sentence = sentence[:start - length_diff] + suggestion + sentence[end - length_diff:]
        length_diff += len(original) - len(suggestion)
    assert original_length == len(sentence.split()), sentence
    return sentence

In [8]:
def correct_sentence_spelling(sentence):
    headers = { 'Content-Type': 'application/json', 'token': TOKEN }
    body = {'sentence': sentence}
    response = requests.post(BASE_URL, headers=headers, json=body)
    return format_response(response, sentence)

In [9]:
def text_preprocess(text):
    text = remove_HTML(text)
    text = convert_unicode(text) 
    # text = standardize_sentence_typing(text)
    text = normalize_acronyms(text)
    text = correct_sentence_spelling(text)
    text = word_segmentation(text) # When use PhoBERT
    text = remove_unnecessary_characters(text)
    return text

In [10]:
def correct_reviews_spelling(reviews):
    with dummy.Pool(cpu_count() * 8) as thread_pool:
        data = list(tqdm(
            thread_pool.imap_unordered(
                lambda x: (x[0], text_preprocess(x[1])), 
                enumerate(reviews)
            ), 
            total = len(reviews)
        ))
    data.sort(key=lambda tup: tup[0])
    return data

# Save preprocessed data

In [11]:
train_reviews = correct_reviews_spelling(df_train.Review)
df_train.Review = pd.DataFrame(train_reviews)[1]
df_train.to_csv('dataset/VLSP_Hotel/ViettelAI_hotel_train_cased.csv', index=False)
df_train.head(3)

,Review,FACILITIES#CLEANLINESS,FACILITIES#COMFORT,FACILITIES#DESIGN&FEATURES,FACILITIES#GENERAL,FACILITIES#MISCELLANEOUS,FACILITIES#PRICES,FACILITIES#QUALITY,FOOD&DRINKS#MISCELLANEOUS,FOOD&DRINKS#PRICES,FOOD&DRINKS#QUALITY,FOOD&DRINKS#STYLE&OPTIONS,HOTEL#CLEANLINESS,HOTEL#COMFORT,HOTEL#DESIGN&FEATURES,HOTEL#GENERAL,HOTEL#MISCELLANEOUS,HOTEL#PRICES,HOTEL#QUALITY,LOCATION#GENERAL,ROOM_AMENITIES#CLEANLINESS,ROOM_AMENITIES#COMFORT,ROOM_AMENITIES#DESIGN&FEATURES,ROOM_AMENITIES#GENERAL,ROOM_AMENITIES#MISCELLANEOUS,ROOM_AMENITIES#PRICES,ROOM_AMENITIES#QUALITY,ROOMS#CLEANLINESS,ROOMS#COMFORT,ROOMS#DESIGN&FEATURES,ROOMS#GENERAL,ROOMS#MISCELLANEOUS,ROOMS#PRICES,ROOMS#QUALITY,SERVICE#GENERAL
0,Rộng_rãi khách_sạn mới nhưng rất vắng Các dịch...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Địa_điểm thuận_tiện trong vòng bán_kính 1 5 km...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Phục_vụ view đẹp vị_trí,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [12]:
val_reviews = correct_reviews_spelling(df_val.Review)
df_val.Review = pd.DataFrame(val_reviews)[1]
df_val.to_csv('dataset/VLSP_Hotel/ViettelAI_hotel_val_cased.csv', index=False)
df_val.head(3)

,Review,FACILITIES#CLEANLINESS,FACILITIES#COMFORT,FACILITIES#DESIGN&FEATURES,FACILITIES#GENERAL,FACILITIES#MISCELLANEOUS,FACILITIES#PRICES,FACILITIES#QUALITY,FOOD&DRINKS#MISCELLANEOUS,FOOD&DRINKS#PRICES,FOOD&DRINKS#QUALITY,FOOD&DRINKS#STYLE&OPTIONS,HOTEL#CLEANLINESS,HOTEL#COMFORT,HOTEL#DESIGN&FEATURES,HOTEL#GENERAL,HOTEL#MISCELLANEOUS,HOTEL#PRICES,HOTEL#QUALITY,LOCATION#GENERAL,ROOM_AMENITIES#CLEANLINESS,ROOM_AMENITIES#COMFORT,ROOM_AMENITIES#DESIGN&FEATURES,ROOM_AMENITIES#GENERAL,ROOM_AMENITIES#MISCELLANEOUS,ROOM_AMENITIES#PRICES,ROOM_AMENITIES#QUALITY,ROOMS#CLEANLINESS,ROOMS#COMFORT,ROOMS#DESIGN&FEATURES,ROOMS#GENERAL,ROOMS#MISCELLANEOUS,ROOMS#PRICES,ROOMS#QUALITY,SERVICE#GENERAL
0,Chưa có thang_máy Chưa chấp_nhận thanh_toán bằ...,0,0,2,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1
1,Chúng_tôi không cảm_thấy thoải_mái vì ở chỉ 1 ...,0,2,0,0,0,0,2,0,0,1,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,Nhân_viên rất nhiệt_tình thân_thiện luôn giúp_...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1


In [13]:
test_reviews = correct_reviews_spelling(df_test.Review)
df_test.Review = pd.DataFrame(test_reviews)[1]
df_test.to_csv('dataset/VLSP_Hotel/ViettelAI_hotel_test_cased.csv', index=False)
df_test.head(3)

,Review,FACILITIES#CLEANLINESS,FACILITIES#COMFORT,FACILITIES#DESIGN&FEATURES,FACILITIES#GENERAL,FACILITIES#MISCELLANEOUS,FACILITIES#PRICES,FACILITIES#QUALITY,FOOD&DRINKS#MISCELLANEOUS,FOOD&DRINKS#PRICES,FOOD&DRINKS#QUALITY,FOOD&DRINKS#STYLE&OPTIONS,HOTEL#CLEANLINESS,HOTEL#COMFORT,HOTEL#DESIGN&FEATURES,HOTEL#GENERAL,HOTEL#MISCELLANEOUS,HOTEL#PRICES,HOTEL#QUALITY,LOCATION#GENERAL,ROOM_AMENITIES#CLEANLINESS,ROOM_AMENITIES#COMFORT,ROOM_AMENITIES#DESIGN&FEATURES,ROOM_AMENITIES#GENERAL,ROOM_AMENITIES#MISCELLANEOUS,ROOM_AMENITIES#PRICES,ROOM_AMENITIES#QUALITY,ROOMS#CLEANLINESS,ROOMS#COMFORT,ROOMS#DESIGN&FEATURES,ROOMS#GENERAL,ROOMS#MISCELLANEOUS,ROOMS#PRICES,ROOMS#QUALITY,SERVICE#GENERAL
0,Ga giường không sạch nhân_viên quen dọn phòng ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,nhân_viên nhiệt_tình phòng ở sạch_sẽ tiện_nghi...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3,1,0,0,0,0,0,1
2,Đi bộ ra biển gần tiện đi_lại Phòng view biển ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0
